In [1]:
from sympy import symbols, sqrt, Matrix, simplify, diff, init_printing, lambdify
import numpy as np
import cvxopt

# Enable LaTeX rendering in Jupyter
init_printing()

# Define variables
x1, x2, y1, y2, r, k = symbols('x1 x2 y1 y2 r k')
u1x, u1y, u2x, u2y = symbols('u1x u1y u2x u2y')

In [2]:
# Define h(x) function
h = sqrt((x1-x2)**2 + (y1-y2)**2) - r
# Define f(x) and g(x) as matrices
f = Matrix([0, 0])
g = Matrix([
    [1, 0],  # Position derivatives (p1, p2, p3) do not depend on control inputs
    [0, 1]
])
u1 = Matrix([u1x, u1y])
u2 = Matrix([u2x, u2y])
state_vars1 = Matrix([x1, y1])
state_vars2 = Matrix([x2, y2])


In [3]:
# Compute Lf(h) (Lie derivative of h along f)
grad_h1 = h.diff(state_vars1).T
grad_h2 = h.diff(state_vars2).T
Lf1_h = grad_h1 * f
Lf2_h = grad_h2 * f
display(Lf1_h)
display(Lf2_h)

[0]

[0]

In [4]:
# Compute Lgh (lie derivative of h along g)
Lg1_h = grad_h1 * g
Lg2_h = grad_h2 * g
display(Lg1_h)
display(Lg2_h)

⎡          x₁ - x₂                       y₁ - y₂           ⎤
⎢────────────────────────────  ────────────────────────────⎥
⎢   _________________________     _________________________⎥
⎢  ╱          2            2     ╱          2            2 ⎥
⎣╲╱  (x₁ - x₂)  + (y₁ - y₂)    ╲╱  (x₁ - x₂)  + (y₁ - y₂)  ⎦

⎡          -x₁ + x₂                      -y₁ + y₂          ⎤
⎢────────────────────────────  ────────────────────────────⎥
⎢   _________________________     _________________________⎥
⎢  ╱          2            2     ╱          2            2 ⎥
⎣╲╱  (x₁ - x₂)  + (y₁ - y₂)    ╲╱  (x₁ - x₂)  + (y₁ - y₂)  ⎦

In [8]:
sep = 2.0
alpha = 2.0

a = 0.1
beta = 1

c1 = 1
c2 = 1000

gamma1 = a*c1**beta
gamma2 = a*c2**beta

print(f"Gamma 1: {gamma1}")
print(f"Gamma 2: {gamma2}")

Lg1_h_func = lambdify((x1, y1, x2, y2, r), Lg1_h)
Lg2_h_func = lambdify((x1, y1, x2, y2, r), Lg2_h)
h_func = lambdify((x1, y1, x2, y2, r), h)

A1 = Lg1_h_func(-1, 0.01, 1, -0.01, sep)
A2 = Lg2_h_func(-1, 0.01, 1, -0.01, sep)
# print(A1)
# print(A2)
b1 = -alpha*h_func(-1, 0.01, 1, -0.01, sep)
b2 = -alpha*h_func(-1, 0.01, 1, -0.01, sep)

b = b1 + b2 + alpha*h_func(-1, 0.1, 1, -0.1, sep)

u1_nom = np.array([10, 0])
u2_nom = np.array([-10, 0])

k1_hat = A1 * u1_nom / b
k2_hat = A2 * u2_nom / b

print(k1_hat)
print(k2_hat)
print(k1_hat+k2_hat)


Gamma 1: 0.1
Gamma 2: 100.0
[[-511.47661624    0.        ]]
[[-511.47661624   -0.        ]]
[[-1022.95323248     0.        ]]


In [9]:
print(A1.shape)
print(u1_nom.shape)
print(A1*u1_nom)

(1, 2)
(2,)
[[-9.99950004  0.        ]]


In [10]:
n = 2

gamma_array = np.array([gamma1, gamma2])
k_hat_array = np.array([k1_hat[0], k2_hat[0]])

print(gamma_array)
print(k_hat_array)

# Objective matrix (H) and vector (f) for the quadratic form
H = 2 * np.diag(gamma_array)  # Quadratic terms - diagonal matrix (size n x n)
f = -2 * gamma_array * k_hat_array  # Linear terms (from the norm)

print(f.shape)

# Convert f into a cvxopt matrix and reshape it into a column vector (size n x 1)
f = cvxopt.matrix(f.astype(float))  # Ensure f is of type float

# Constraints matrix (A) and vector (b)
A = np.ones((1, n))  # Sum of a_i's must be at least 1
b = np.array([1.0])

# Bounds on a_i
# Assuming a_i >= 0 for all i, hence the lower bound is 0
G = -np.eye(n)  # Negative identity matrix for a_i >= 0
h = np.zeros(n)

# Convert all variables into cvxopt matrices, ensuring correct float type
H = cvxopt.matrix(H.astype(float))  # Ensure H is of type float
A = cvxopt.matrix(A.astype(float))  # Ensure A is of type float
b = cvxopt.matrix(b.astype(float))  # Ensure b is of type float
G = cvxopt.matrix(G.astype(float))  # Ensure G is of type float
h = cvxopt.matrix(h.astype(float))  # Ensure h is of type float

# Solve the QP problem using cvxopt.solvers.qp
sol = cvxopt.solvers.qp(H, f, None, None, A, b)

# Get the solution
a_i_values = np.array(sol['x']).flatten()

print("Optimal a_i values:", a_i_values)

[  0.1 100. ]
[[-511.47661624    0.        ]
 [-511.47661624   -0.        ]]
(2, 2)


TypeError: 'q' must be a 'd' matrix with one column

In [ ]:
u1_new = b * a_i_values[0] / A1
u2_new = b * a_i_values[1] / A2

print(u1_new)
print(u2_new)